In [ ]:
import nlpaug.augmenter.word as naw
import pandas as pd

df = pd.read_csv("train.csv")
df_filtered = df[df["stereotype"] == 1].copy()
df_filtered["id"] = df_filtered["id"].apply(lambda x: x + "_a")
df_filtered["comment_id"] = df_filtered["comment_id"].apply(lambda x: x + "_a")

back_translation_aug = naw.BackTranslationAug(
    from_model_name='Helsinki-NLP/opus-mt-es-en', 
    to_model_name='Helsinki-NLP/opus-mt-en-es',
)

back_translate_text = back_translation_aug.augment(list(df_filtered['text'].values))
df_filtered['text'] = back_translate_text

df2 = pd.concat([df, df_filtered], axis=0, ignore_index=True)
print(df.shape)
print(df_filtered.shape)
print(df2.shape)

df2.to_csv("train_augmented.csv")

In [ ]:
import pandas as pd

#Read original CSV file
original_train_file = "train.csv"
df = pd.read_csv(original_train_file)

df["test_case"] = "DETEST-Dis" #Add a new column

df["stereotype"] = df["stereotype"].apply(lambda x: "Stereotype" if x == 1 else "NoStereotype")
df["stereotype_soft"] = df["stereotype_soft"].apply(lambda x:{"Stereotype": x, "NoStereotype": 1 - x})

df["implicit"] = df["implicit"].apply(lambda x: "Implicit" if x==1 else "Explicit")
df["implicit_soft"] = df["implicit_soft"].apply(lambda x: {"Implicit": x, "Explicit": 1 - x})

def json_eval_file(df, col_name, json_file_name):
    df["value"] = df[col_name].values
    df[["id", "test_case", "value"]].to_json(json_file_name, orient='records', indent=4)
filtered_df = df[df["stereotype"] == "Stereotype"]


#Stereotype
json_eval_file(df, "stereotype", "train_stereotype_hard.json")
json_eval_file(df, "stereotype_soft", "train_stereotype_soft.json")
#Implicit
json_eval_file(filtered_df, "implicit", "train_implicit_hard.json")
json_eval_file(filtered_df, "implicit_soft", "train_implicit_soft.json")